# 1 初始化

## 1.1 导入包、超参数定义

In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from tqdm import tqdm

import logging
log_fmt = "[%(asctime)s] %(levelname)s: %(message)s"
logging.basicConfig(format=log_fmt, level=logging.INFO)
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "3"


import gzip
import pickle
base_path = '/home/zengrui/datasets/190829_Kanshan_zjfx'

def save_param(param, param_name):
    with open(f'{base_path}/temp/{param_name}.txt', 'wb') as f:
        pickle.dump(param, f)
        
def read_param(param_name):
    with open(f'{base_path}/temp/{param_name}.txt', 'rb') as f:
        return pickle.load(f)

def gload(file_name):
    with gzip.GzipFile(f'{base_path}/temp/{file_name}.gzip', 'rb') as f:
        return pickle.load(f)

def gdump(obj, file_name):
    with gzip.GzipFile(f'{base_path}/temp/{file_name}.gzip', 'wb') as f:
        pickle.dump(obj, f)

# 2 问题词偏好挖掘

## 2.1 加载数据

需要加载的数据：

* `question`: qid, title_t2, desc_t2
* `answer`: uid, qid
* `invite`: qid, uid

Load `invite`, `invite_evaluate`.

> `dtype`: obj->int, speed up **50x**.

In [2]:
train = pd.read_csv(f'{base_path}/invite_info_0926.txt', sep='\t', header=None)
train.columns = ['qid', 'uid', 'dt', 'label']
# test = pd.read_csv(f'{base_path}/invite_info_evaluate_1_0926.txt', sep='\t', header=None)
test = pd.read_csv(f'{base_path}/invite_info_evaluate_2_0926.txt', sep='\t', header=None)
test.columns = ['qid', 'uid', 'dt']

def extract_day(s):
    return s.apply(lambda x: int(x.split('-')[0][1:]))

train['day'] = extract_day(train['dt'])
train = train[train['day'] > 3860]

del train['dt'], train['label'], train['day'], test['dt']

# 优化id为int
train['qid'] = pd.to_numeric(train['qid'].apply(lambda x: x[1:]))
train['uid'] = pd.to_numeric(train['uid'].apply(lambda x: x[1:]))
test['qid'] = pd.to_numeric(test['qid'].apply(lambda x: x[1:]))
test['uid'] = pd.to_numeric(test['uid'].apply(lambda x: x[1:]))

train_size, test_size = len(train), len(test)

logging.info('train %s, test %s', train.shape, test.shape)
logging.info('columns: %s', train.columns)
logging.info('dtypes: %s', train.dtypes)

[2019-12-17 09:30:22,227] INFO: train (2593669, 2), test (1141718, 2)
[2019-12-17 09:30:22,230] INFO: columns: Index(['qid', 'uid'], dtype='object')
[2019-12-17 09:30:22,232] INFO: dtypes: qid    int64
uid    int64
dtype: object


Load `question`.

In [3]:
ques = pd.read_csv(f'{base_path}/question_info_0926.txt', header=None, sep='\t')
ques.columns = ['qid', 'q_dt', 'title_t1', 'title_t2', 'desc_t1', 'desc_t2', 'topic']

for i in ['q_dt', 'title_t1', 'desc_t1']:
    del ques[i]

# 优化id为int
ques['qid'] = pd.to_numeric(ques['qid'].apply(lambda x: x[1:]))
ques['title_t2'] = ques['title_t2'].astype(str)
ques['desc_t2'] = ques['desc_t2'].astype(str)
ques['topic'] = ques['topic'].astype(str)

logging.info('ques: %s', ques.shape)
logging.info('columns: %s', ques.columns)
logging.info('dtypes: %s', ques.dtypes)

[2019-12-17 09:30:42,102] INFO: ques: (1829900, 4)
[2019-12-17 09:30:42,104] INFO: columns: Index(['qid', 'title_t2', 'desc_t2', 'topic'], dtype='object')
[2019-12-17 09:30:42,105] INFO: dtypes: qid          int64
title_t2    object
desc_t2     object
topic       object
dtype: object


Load `answer`.

In [4]:
ans = pd.read_csv(f'{base_path}/answer_info_0926.txt', header=None, sep='\t')
ans.columns = ['aid', 'qid', 'uid', 'ans_dt', 'ans_t1', 'ans_t2', 'is_good', 'is_rec', 'is_dest', 'has_img',
               'has_video', 'word_count', 'reci_cheer', 'reci_uncheer', 'reci_comment', 'reci_mark', 'reci_tks',
               'reci_xxx', 'reci_no_help', 'reci_dis']

for i in ['aid', 'ans_dt', 'ans_t1', 'ans_t2', 'is_good', 'is_rec', 'is_dest', 
          'has_img', 'has_video', 'word_count', 'reci_cheer', 'reci_uncheer', 
          'reci_comment', 'reci_mark', 'reci_tks', 'reci_xxx', 'reci_no_help', 
          'reci_dis']:
    del ans[i]

# 优化id为int
ans['qid'] = pd.to_numeric(ans['qid'].apply(lambda x: x[1:]))
ans['uid'] = pd.to_numeric(ans['uid'].apply(lambda x: x[1:]))

logging.info("ans: %s", ans.shape)
logging.info("columns: %s", ans.columns)
logging.info("dtypes: %s", ans.dtypes)

[2019-12-17 09:32:14,247] INFO: ans: (4513735, 2)
[2019-12-17 09:32:14,248] INFO: columns: Index(['qid', 'uid'], dtype='object')
[2019-12-17 09:32:14,249] INFO: dtypes: qid    int64
uid    int64
dtype: object


## 2.2 提取数据

In [5]:
# 合并 train & test -> data
data = pd.concat((train, test), axis=0, sort=True)
del train, test

len(set(data['uid']))

1051067

### 2.2.1 创建uhist表

创建 **uid - 个人历史回答问题的标题拼接, 个人历史回答的描述拼接, 个人历史回答的topic拼接** 表 `uhist` (user_history)。

In [6]:
uhist = pd.DataFrame()
uhist['uid'] = pd.Series(list(set(data['uid'])))
print(uhist.dtypes)

uid    int64
dtype: object


读取每个人历史回答词拼接。（注意：速度上，`联表 + groupby` 计算远比 `apply` 快。）

In [7]:
# 联表 uhist - ans - ques
uhist = uhist.merge(ans, on='uid', how='left')
uhist = uhist.merge(ques, on='qid', how='left')
del uhist['qid'], ans

uhist = uhist.fillna('')  # 化NaN为''

print(len(uhist))
print(uhist.columns)

4627363
Index(['uid', 'title_t2', 'desc_t2', 'topic'], dtype='object')


In [8]:
# 统计每个人所有 ques 的字符串拼接
uhist = pd.DataFrame(
    uhist.groupby('uid')['title_t2'].apply(
        lambda x: ','.join(list(filter(lambda x: x != '-1', x)))
    )
).merge(
    pd.DataFrame(
        uhist.groupby('uid')['desc_t2'].apply(
            lambda x: ','.join(list(filter(lambda x: x != '-1', x)))
        )
    ), on='uid', how='left'
).merge(
    pd.DataFrame(
        uhist.groupby('uid')['topic'].apply(
            lambda x: ','.join(list(filter(lambda x: x != '-1', x)))
        )
    ), on='uid', how='left'
)
uhist

,title_t2,desc_t2,topic
uid,,,
2786,"W13983,W35802,W5618,W1201,W4199,W16462,W840,W2...","W894,W188,W126,W17765,W1761,W848,W5019,W2376,W...","T421,T421,T1953,T1045,T1953,T27,T185"
9838,,,
15476,"W11155,W10203,W2139,W856,W51,W184,W6135,W1276,...","W705,W14650,W51,W4274,W123,W11155,W938,W121536...","T2070,T1725,T1649,T548,T2070"
16689,"W31273,W33874,W1222,W29020,W81814,W38692,W1241...","W546,W233,W270,W79,W412,W466,W522,W105161,W200...","T4642,T14584,T430,T2306,T126,T13,T15991"
24521,"W973,W659,W5104,W26102,W3931",W689,"T166,T55"
...,...,...,...
4294958617,,,
4294959078,"W1969,W12985,W43374,W827,W35689,W490312,W73280...","W19761,W370,W30329,W41726,W3865,W93971,W233,W1...","T7130,T726,T57,T125,T7085,T3483,T172,T10696,T1..."
4294962025,"W11550,W1684,W822,W622,W1411,W126,W6675,W227,W...","W1684,W11550,W1157,W822,W622,W319,W448,W5771,W...","T208,T439,T2646,T19310,T313,T16,T5027,T548"


In [9]:
# 赶快存储 uhist
gdump(uhist, 'uhist_v6_2')

### 2.2.2 数据汇集到data表

In [10]:
# ques, uhist 数据汇集到 data
data = data.merge(ques, on='qid', how='left')
data = data.merge(uhist, on='uid', how='left')

data.columns = ['qid', 'uid', 'title_t2', 'desc_t2', 'topic', 'hist_title_t2', 'hist_desc_t2', 'hist_topic']
logging.info(data.columns)

data['title_t2'].replace('-1', '', inplace=True)
data['desc_t2'].replace('-1', '', inplace=True)
data['topic'].replace('-1', '', inplace=True)

del ques, uhist

[2019-12-17 09:41:43,519] INFO: Index(['qid', 'uid', 'title_t2', 'desc_t2', 'topic', 'hist_title_t2',
       'hist_desc_t2', 'hist_topic'],
      dtype='object')


In [11]:
data

,qid,uid,title_t2,desc_t2,topic,hist_title_t2,hist_desc_t2,hist_topic
0,2166419046,401693808,"W11058,W272,W2202,W431,W951,W243,W3828,W3037,W263","W3828,W14870,W3037,W2450,W692,W1688,W53547,W24...","T456,T112,T9566,T5310",,,
1,604029601,2317670257,"W31489,W9218,W3440,W243","W3046,W221,W3440,W730,W31489,W9218,W1278,W415","T6090,T2156,T97,T456","W272,W263,W4534,W7742,W973,W3688,W3973,W258,W6...","W272,W4534,W7742,W3738,W1277,W111,W2425,W41838...","T10,T10186,T42650,T184,T1203,T4454,T5649,T5122"
2,2443223942,3544409350,"W554,W28208,W396,W51",,"T26,T76,T17",,,
3,795459266,2818659842,"W8705,W5085,W185",,"T8430,T6916,T7356,T24271,T5817","W886,W188,W19433,W16846,W105,W1222,W6481,W2652...","W638,W886,W894,W188,W11222,W105,W715,W3995,W29...","T426,T1297,T7023,T9779,T70,T407,T7710,T1749,T1..."
4,110462128,848334644,"W9097,W32077,W5634","W9097,W9335,W23221,W788,W6778,W37109,W12685,W1...","T561,T2618","W1891,W9097,W58494,W140823,W2486,W1276,W351,W9...","W1090,W5074,W1914,W1968,W2126,W3495,W6658,W152...","T561,T6197,T561,T252,T27,T561,T8546,T561,T2618..."
...,...,...,...,...,...,...,...,...
3735382,2523437455,2490844425,"W192,W675,W396,W894,W188,W1961",,"T144,T17,T27",,,
3735383,3958271698,1271605368,"W2928,W345,W1340,W3103,W2001,W1840,W51,W641,W2...",,"T251,T671,T138","W316,W3387,W668,W16207,W2191,W2819,W51,W3995",,"T1466,T17238,T3,T7592,T426"
3735384,4242683638,1452284822,"W8255,W876,W9047,W70697,W457,W565,W25414,W4158",,"T456,T378,T24,T3857","W1222,W3897,W221,W4692,W876,W9047,W3860",,"T456,T1015,T3857"
3735385,1116532200,521953190,"W2599,W272,W149","W2165,W429,W809,W2599,W272,W149,W708,W429,W259...","T47,T50,T62,T901,T10",,,


## 2.3 挖掘数据

挖掘 ['title_t2', 'desc_t2'] 两者中的以下数据：

* a. ques的word中曾经出现过个种 `len(set(word) & set(hist_word))`
* b. ques中曾经word占ques所有word比例 `a / len(set(word))`
* c. ques中曾经word占所有曾经word比例 `a / len(set(hist_word))`
* d. ques的word中曾经出现过个次 `len([x for x in word if x in (set(word) & set(hist_word))])`
* e. ques的word中曾经出现过个次占所有曾经出现过个次比例 `d / len(hist_word)`

```python
a = len(set(coauthor) & set(history_coauthor))
b = a / len(set(coauthor))
c = a / len(set(history_coauthor))
d = len([x for x in coauthor if x in (set(coauthor) & set(history_coauthor))])
e = d / len(history_coauthor)
```

In [14]:
# 一列 apply 生成多列的方法
# def tmp(x):
#     return x, x*2, x*3

# a, b, c = zip(*data[:1000]['uid'].apply(tmp))

In [12]:
pbar = tqdm(total=len(data))


def foo(word, hist_word):
    '''输入 ques_word 和 hist_word, 输出 a, b, c, d, e'''
    pbar.update()
    if 0 in [len(word), len(hist_word)]:  # 排除空输入
        return 0, 0, 0, 0, 0
    sw, shw = set(word), set(hist_word)
    lsw, lshw = len(sw), len(shw)
    and_set = sw & shw
    a = len(and_set)
    b = a / lsw
    c = a / lshw
    d = len([x for x in word if x in and_set])
    e = d / len(hist_word)
    return a, b, c, d, e


data['a'], data['b'], data['c'], data['d'], data['e'] = zip(*data[['title_t2', 'hist_title_t2']].apply(
    lambda r: foo(r['title_t2'], r['hist_title_t2']), axis=1))
pbar.close()

pbar = tqdm(total=len(data))
data['aa'], data['bb'], data['cc'], data['dd'], data['ee'] = zip(*data[['desc_t2', 'hist_desc_t2']].apply(
    lambda r: foo(r['desc_t2'], r['hist_desc_t2']), axis=1))
pbar.close()

pbar = tqdm(total=len(data))
data['aaa'], data['bbb'], data['ccc'], data['ddd'], data['eee'] = zip(*data[['topic', 'hist_topic']].apply(
    lambda r: foo(r['topic'], r['hist_topic']), axis=1))
pbar.close()

data

100%|██████████| 3735387/3735387 [01:38<00:00, 38088.95it/s]


,qid,uid,title_t2,desc_t2,topic,hist_title_t2,hist_desc_t2,hist_topic,a,b,...,aa,bb,cc,dd,ee,aaa,bbb,ccc,ddd,eee
0,2166419046,401693808,"W11058,W272,W2202,W431,W951,W243,W3828,W3037,W263","W3828,W14870,W3037,W2450,W692,W1688,W53547,W24...","T456,T112,T9566,T5310",,,,0,0.000000,...,0,0.0,0.0,0,0.000000,0,0.000000,0.000000,0,0.000000
1,604029601,2317670257,"W31489,W9218,W3440,W243","W3046,W221,W3440,W730,W31489,W9218,W1278,W415","T6090,T2156,T97,T456","W272,W263,W4534,W7742,W973,W3688,W3973,W258,W6...","W272,W4534,W7742,W3738,W1277,W111,W2425,W41838...","T10,T10186,T42650,T184,T1203,T4454,T5649,T5122",9,1.000000,...,12,1.0,1.0,45,0.173077,9,0.900000,0.818182,19,0.413043
2,2443223942,3544409350,"W554,W28208,W396,W51",,"T26,T76,T17",,,,0,0.000000,...,0,0.0,0.0,0,0.000000,0,0.000000,0.000000,0,0.000000
3,795459266,2818659842,"W8705,W5085,W185",,"T8430,T6916,T7356,T24271,T5817","W886,W188,W19433,W16846,W105,W1222,W6481,W2652...","W638,W886,W894,W188,W11222,W105,W715,W3995,W29...","T426,T1297,T7023,T9779,T70,T407,T7710,T1749,T1...",7,1.000000,...,0,0.0,0.0,0,0.000000,12,1.000000,1.000000,30,0.245902
4,110462128,848334644,"W9097,W32077,W5634","W9097,W9335,W23221,W788,W6778,W37109,W12685,W1...","T561,T2618","W1891,W9097,W58494,W140823,W2486,W1276,W351,W9...","W1090,W5074,W1914,W1968,W2126,W3495,W6658,W152...","T561,T6197,T561,T252,T27,T561,T8546,T561,T2618...",10,1.000000,...,12,1.0,1.0,56,0.012267,7,1.000000,0.583333,10,0.016103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3735382,2523437455,2490844425,"W192,W675,W396,W894,W188,W1961",,"T144,T17,T27",,,,0,0.000000,...,0,0.0,0.0,0,0.000000,0,0.000000,0.000000,0,0.000000
3735383,3958271698,1271605368,"W2928,W345,W1340,W3103,W2001,W1840,W51,W641,W2...",,"T251,T671,T138","W316,W3387,W668,W16207,W2191,W2819,W51,W3995",,"T1466,T17238,T3,T7592,T426",10,0.909091,...,0,0.0,0.0,0,0.000000,9,1.000000,0.818182,14,0.538462
3735384,4242683638,1452284822,"W8255,W876,W9047,W70697,W457,W565,W25414,W4158",,"T456,T378,T24,T3857","W1222,W3897,W221,W4692,W876,W9047,W3860",,"T456,T1015,T3857",10,0.909091,...,0,0.0,0.0,0,0.000000,8,0.888889,0.800000,18,1.125000
3735385,1116532200,521953190,"W2599,W272,W149","W2165,W429,W809,W2599,W272,W149,W708,W429,W259...","T47,T50,T62,T901,T10",,,,0,0.000000,...,0,0.0,0.0,0,0.000000,0,0.000000,0.000000,0,0.000000


## 2.4 压缩数据

In [13]:
# 删除没用的列
# for i in ['title_t2', 'hist_title_t2', 'desc_t2', 'hist_desc_t2']:
#     del data[i]
    
t = data.dtypes
for x in t[t == 'int64'].index:
    data[x] = data[x].astype('int32')
for x in t[t == 'float64'].index:
    data[x] = data[x].astype('float32')
    
feature_cols_v6 = [x for x in data.columns if x not in ['uid', 'qid', 'title_t2', 'hist_title_t2', 
                                                          'desc_t2', 'hist_desc_t2', 'topic', 'hist_topic']]
logging.info('feature_cols: %s', feature_cols_v6)

[2019-12-17 09:47:41,948] INFO: feature_cols: ['a', 'b', 'c', 'd', 'e', 'aa', 'bb', 'cc', 'dd', 'ee', 'aaa', 'bbb', 'ccc', 'ddd', 'eee']


In [14]:
# 赶快存变量
X_train_all_v6 = data.iloc[:train_size][feature_cols_v6]
test_v6 = data.iloc[train_size:][feature_cols_v6]
test_v6 = test_v6.reset_index()

gdump(X_train_all_v6, 'X_train_all_v6')
gdump(test_v6, 'test_v6_2')
gdump(feature_cols_v6, 'feature_cols_v6')

In [15]:
X_train_all_v6

,a,b,c,d,e,aa,bb,cc,dd,ee,aaa,bbb,ccc,ddd,eee
0,0,0.0,0.000000,0,0.000000,0,0.0,0.0,0,0.000000,0,0.0,0.000000,0,0.000000
1,9,1.0,0.750000,23,0.209091,12,1.0,1.0,45,0.173077,9,0.9,0.818182,19,0.413043
2,0,0.0,0.000000,0,0.000000,0,0.0,0.0,0,0.000000,0,0.0,0.000000,0,0.000000
3,7,1.0,0.583333,16,0.064257,0,0.0,0.0,0,0.000000,12,1.0,1.000000,30,0.245902
4,10,1.0,0.833333,18,0.008751,12,1.0,1.0,56,0.012267,7,1.0,0.583333,10,0.016103
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2593664,0,0.0,0.000000,0,0.000000,0,0.0,0.0,0,0.000000,0,0.0,0.000000,0,0.000000
2593665,11,1.0,0.916667,30,0.120968,0,0.0,0.0,0,0.000000,8,1.0,0.666667,14,0.140000
2593666,10,1.0,0.833333,20,0.080645,12,1.0,1.0,97,1.000000,7,1.0,0.583333,15,0.150000
2593667,11,1.0,0.916667,36,0.014057,0,0.0,0.0,0,0.000000,3,1.0,0.250000,3,0.002292


In [19]:
del data, test_v6

# uhist数据训练

## 导入包、超参数定义

In [2]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten

import os
import tensorflow as tf
from keras.backend import set_session
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8  # 占用GPU百分比
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config)) # 此处不同

BATCH_SIZE = 8192
EPOCHS = 5
LR = 0.01

callbacks=[
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', 
                                         factor=0.1, 
                                         patience=3, 
                                         verbose=1, 
                                         mode='auto', 
                                         min_delta=0.0001, 
                                         cooldown=0, 
                                         min_lr=0),  # 当指标变化小时，减少学习率
]

Using TensorFlow backend.


In [3]:
def get_easy_net(input_dim, optimizer='rmsprop', loss='binary_crossentropy'):
    m = Sequential()

    m.add(Dense(256, activation='tanh', input_shape=(input_dim,)))
    m.add(Dropout(0.3))
    m.add(Dense(512, activation='relu'))
    m.add(Dropout(0.3))
    m.add(Dense(512, activation='tanh'))
    m.add(Dropout(0.3))
    m.add(Dense(256, activation='linear'))
    m.add(Dense(1, activation='sigmoid'))

    m.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

    return m

## 数据处理

准备数据。

In [21]:
X_train = gload('X_train_all_v6')
y_train = gload('y_train_all')

y_train

0          0.0
1          0.0
2          0.0
3          0.0
4          1.0
          ... 
2593664    0.0
2593665    0.0
2593666    1.0
2593667    1.0
2593668    0.0
Name: label, Length: 2593669, dtype: float32

归一化数据。

In [22]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
x = X_train.values
x = mms.fit_transform(x)
X_train = pd.DataFrame(x)
X_train.columns = ['a', 'b', 'c', 'd', 'e', 'aa', 'bb', 'cc', 'dd', 'ee', 'aaa', 'bbb', 'ccc', 'ddd', 'eee']
del x
X_train[:5]

,a,b,c,d,e,aa,bb,cc,dd,ee,aaa,bbb,ccc,ddd,eee
0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.750000,1.0,0.750000,0.097872,0.017139,1.0,1.0,1.0,0.007606,0.000648,0.750000,0.9,0.818182,0.322034,0.082609
2,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,0.583333,1.0,0.583333,0.068085,0.005267,0.0,0.0,0.0,0.000000,0.000000,1.000000,1.0,1.000000,0.508475,0.049180
4,0.833333,1.0,0.833333,0.076596,0.000717,1.0,1.0,1.0,0.009466,0.000046,0.583333,1.0,0.583333,0.169492,0.003221


过采样均衡数据。

In [43]:
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

logging.info('y_train distribution is %s' % Counter(y_train))
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
logging.info('y_resampled distribution is %s' % Counter(y_resampled))

# del X_train, y_train

[2019-12-16 19:44:58,918] INFO: y_train distribution is Counter({0.0: 2178656, 1.0: 415013})
[2019-12-16 19:45:03,538] INFO: y_resampled distribution is Counter({0.0: 2178656, 1.0: 2178656})


手动shuffle数据。

In [44]:
X_resampled = X_resampled.values
y_resampled = y_resampled.values
np.random.seed(100)
np.random.shuffle(X_resampled)
np.random.seed(100)
np.random.shuffle(y_resampled)

## NN训练

### 网络1

In [25]:
model = get_easy_net(input_dim=5)

[2019-12-16 17:16:01,764] WARNING: From /home/zengrui/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [28]:
H = model.fit(X_resampled[:, 0:5], y_resampled, 
              epochs=100, batch_size=BATCH_SIZE, #validation_split=0.1,
              shuffle='batch', callbacks=callbacks, verbose=1)

[2019-12-16 17:16:41,072] WARNING: From /home/zengrui/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6675 - accuracy: 0.5887
Epoch 2/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6480 - accuracy: 0.6111
Epoch 3/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6400 - accuracy: 0.6188
Epoch 4/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6370 - accuracy: 0.6212
Epoch 5/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6352 - accuracy: 0.6221
Epoch 6/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6337 - accuracy: 0.6232
Epoch 7/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6330 - accuracy: 0.6233
Epoch 8/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6319 - accuracy: 0.6240
Epoch 9/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6314 - accuracy: 0.6244
Epoch 10/100
435731

4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6170 - accuracy: 0.6330

Epoch 00074: ReduceLROnPlateau reducing learning rate to 1.0000000656873453e-06.
Epoch 75/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6170 - accuracy: 0.6329
Epoch 76/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6169 - accuracy: 0.6330
Epoch 77/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6170 - accuracy: 0.6329
Epoch 78/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6170 - accuracy: 0.6330
Epoch 79/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6170 - accuracy: 0.6329

Epoch 00079: ReduceLROnPlateau reducing learning rate to 1.0000001111620805e-07.
Epoch 80/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6169 - accuracy: 0.6330
Epoch 81/100
4357312/4357312 [==============================] - 8s 2us/step - l

In [29]:
gdump(model, 'model_nn_col1')

In [63]:
# model = gload('model_nn_col1')

### 网络2

In [30]:
model = get_easy_net(input_dim=5)

In [31]:
H = model.fit(X_resampled[:, 5:10], y_resampled, 
              epochs=100, batch_size=BATCH_SIZE, #validation_split=0.1,
              shuffle='batch', callbacks=callbacks, verbose=1)

Epoch 1/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6844 - accuracy: 0.5603
Epoch 2/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6779 - accuracy: 0.5655
Epoch 3/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6764 - accuracy: 0.5669
Epoch 4/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6756 - accuracy: 0.5677
Epoch 5/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6749 - accuracy: 0.5683
Epoch 6/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6747 - accuracy: 0.5685
Epoch 7/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6743 - accuracy: 0.5688
Epoch 8/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.6740 - accuracy: 0.5692
Epoch 9/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6738 - accuracy: 0.5695
Epoch 10/100
435731

In [32]:
gdump(model, 'model_nn_col2')

### 网络3

In [33]:
model = get_easy_net(input_dim=5)

In [34]:
H = model.fit(X_resampled[:, 10:15], y_resampled, 
              epochs=100, batch_size=BATCH_SIZE, #validation_split=0.1,
              shuffle='batch', callbacks=callbacks, verbose=1)

Epoch 1/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6536 - accuracy: 0.6100
Epoch 2/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6300 - accuracy: 0.6352
Epoch 3/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6233 - accuracy: 0.6422
Epoch 4/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6191 - accuracy: 0.6451
Epoch 5/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6164 - accuracy: 0.6476
Epoch 6/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6143 - accuracy: 0.6492
Epoch 7/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6131 - accuracy: 0.6499
Epoch 8/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6119 - accuracy: 0.6510
Epoch 9/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6112 - accuracy: 0.6514
Epoch 10/100
435731

In [35]:
gdump(model, 'model_nn_col3')

### 网络123

In [36]:
model = get_easy_net(input_dim=15)

In [37]:
H = model.fit(X_resampled, y_resampled, 
              epochs=100, batch_size=BATCH_SIZE, #validation_split=0.1,
              shuffle='batch', callbacks=callbacks, verbose=1)

Epoch 1/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6464 - accuracy: 0.6246
Epoch 2/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6146 - accuracy: 0.6553
Epoch 3/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.6041 - accuracy: 0.6630
Epoch 4/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.5988 - accuracy: 0.6675
Epoch 5/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.5947 - accuracy: 0.6713
Epoch 6/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.5921 - accuracy: 0.6732
Epoch 7/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.5903 - accuracy: 0.6749
Epoch 8/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.5886 - accuracy: 0.6761
Epoch 9/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.5871 - accuracy: 0.6774
Epoch 10/100
435731

In [38]:
gdump(model, 'model_nn_col123')

### 网络1,2,3 output集成网络

In [49]:
model_nn_col1 = gload('model_nn_col1')
model_nn_col2 = gload('model_nn_col2')
model_nn_col3 = gload('model_nn_col3')

c1 = model_nn_col1.predict(X_resampled[:, 0:5], batch_size=BATCH_SIZE)
c2 = model_nn_col2.predict(X_resampled[:, 5:10], batch_size=BATCH_SIZE)
c3 = model_nn_col3.predict(X_resampled[:, 10:15], batch_size=BATCH_SIZE)
x = np.hstack([c1, c2, c3])
x

array([[0.5991044 , 0.6232461 , 0.60203433],
       [0.59318066, 0.569296  , 0.5912592 ],
       [0.4655383 , 0.53929615, 0.15721443],
       ...,
       [0.48663253, 0.6081572 , 0.62659687],
       [0.8994026 , 0.9229114 , 0.5749876 ],
       [0.33083797, 0.45613426, 0.3287472 ]], dtype=float32)

In [50]:
model = get_easy_net(input_dim=3)
H = model.fit(x, y_resampled, 
              epochs=100, batch_size=BATCH_SIZE, #validation_split=0.1,
              shuffle='batch', callbacks=callbacks, verbose=1)

Epoch 1/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.5923 - accuracy: 0.6708
Epoch 2/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.5790 - accuracy: 0.6804
Epoch 3/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.5780 - accuracy: 0.6813
Epoch 4/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.5776 - accuracy: 0.6815
Epoch 5/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.5771 - accuracy: 0.6819
Epoch 6/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.5768 - accuracy: 0.6822
Epoch 7/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.5765 - accuracy: 0.6821
Epoch 8/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.5763 - accuracy: 0.6823
Epoch 9/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.5761 - accuracy: 0.6825
Epoch 10/100
435731

In [51]:
gdump(model, 'model_nn_col1_2_3')

### 删变量

In [52]:
del X_resampled, y_resampled, x, model_nn_col1, model_nn_col2, model_nn_col3, c1, c2, c3

## 计算colsv6_train

In [71]:
X_train = gload('X_train_all_v6')
model_nn_col1 = gload('model_nn_col1')
model_nn_col2 = gload('model_nn_col2')
model_nn_col3 = gload('model_nn_col3')
model_nn_col123 = gload('model_nn_col123')
model_nn_col1_2_3 = gload('model_nn_col1_2_3')

In [72]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
x = X_train.values
x = mms.fit_transform(x)
X_train = pd.DataFrame(x)
X_train.columns = ['a', 'b', 'c', 'd', 'e', 'aa', 'bb', 'cc', 'dd', 'ee', 'aaa', 'bbb', 'ccc', 'ddd', 'eee']
del x
X_train[:5]

,a,b,c,d,e,aa,bb,cc,dd,ee,aaa,bbb,ccc,ddd,eee
0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
1,0.750000,1.0,0.750000,0.097872,0.017139,1.0,1.0,1.0,0.007606,0.000648,0.750000,0.9,0.818182,0.322034,0.082609
2,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
3,0.583333,1.0,0.583333,0.068085,0.005267,0.0,0.0,0.0,0.000000,0.000000,1.000000,1.0,1.000000,0.508475,0.049180
4,0.833333,1.0,0.833333,0.076596,0.000717,1.0,1.0,1.0,0.009466,0.000046,0.583333,1.0,0.583333,0.169492,0.003221


In [73]:
tmp = pd.DataFrame()
tmp['v6_1'] = model_nn_col1.predict(X_train.iloc[:, 0:5].values, batch_size=BATCH_SIZE).reshape(-1)
tmp['v6_2'] = model_nn_col2.predict(X_train.iloc[:, 5:10].values, batch_size=BATCH_SIZE).reshape(-1)
tmp['v6_3'] = model_nn_col3.predict(X_train.iloc[:, 10:15].values, batch_size=BATCH_SIZE).reshape(-1)
tmp['v6_123'] = model_nn_col123.predict(X_train.iloc[:, 0:15].values, batch_size=BATCH_SIZE).reshape(-1)
tmp['v6_1_2_3'] = model_nn_col1_2_3.predict(tmp.iloc[:, 0:3].values, batch_size=BATCH_SIZE).reshape(-1)
tmp

,v6_1,v6_2,v6_3,v6_123,"v6_1,2,3"
0,0.330838,0.456134,0.328747,0.329541,0.329874
1,0.499043,0.517830,0.139592,0.139055,0.125604
2,0.330838,0.456134,0.328747,0.329541,0.329874
3,0.562440,0.456134,0.518578,0.530327,0.575211
4,0.694262,0.569675,0.622583,0.668858,0.696415
...,...,...,...,...,...
2593664,0.330838,0.456134,0.328747,0.329541,0.329874
2593665,0.517593,0.456134,0.511814,0.555835,0.524057
2593666,0.541721,0.964831,0.521709,0.687276,0.963946
2593667,0.672795,0.456134,0.665079,0.666634,0.675585


In [75]:
X_train = pd.concat([X_train, tmp], axis=1)
X_train

,a,b,c,d,e,aa,bb,cc,dd,ee,aaa,bbb,ccc,ddd,eee,v6_1,v6_2,v6_3,v6_123,"v6_1,2,3"
0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.330838,0.456134,0.328747,0.329541,0.329874
1,0.750000,1.0,0.750000,0.097872,0.017139,1.0,1.0,1.0,0.007606,0.000648,0.750000,0.9,0.818182,0.322034,0.082609,0.499043,0.517830,0.139592,0.139055,0.125604
2,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.330838,0.456134,0.328747,0.329541,0.329874
3,0.583333,1.0,0.583333,0.068085,0.005267,0.0,0.0,0.0,0.000000,0.000000,1.000000,1.0,1.000000,0.508475,0.049180,0.562440,0.456134,0.518578,0.530327,0.575211
4,0.833333,1.0,0.833333,0.076596,0.000717,1.0,1.0,1.0,0.009466,0.000046,0.583333,1.0,0.583333,0.169492,0.003221,0.694262,0.569675,0.622583,0.668858,0.696415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2593664,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.330838,0.456134,0.328747,0.329541,0.329874
2593665,0.916667,1.0,0.916667,0.127660,0.009915,0.0,0.0,0.0,0.000000,0.000000,0.666667,1.0,0.666667,0.237288,0.028000,0.517593,0.456134,0.511814,0.555835,0.524057
2593666,0.833333,1.0,0.833333,0.085106,0.006610,1.0,1.0,1.0,0.016396,0.003744,0.583333,1.0,0.583333,0.254237,0.030000,0.541721,0.964831,0.521709,0.687276,0.963946
2593667,0.916667,1.0,0.916667,0.153191,0.001152,0.0,0.0,0.0,0.000000,0.000000,0.250000,1.0,0.250000,0.050847,0.000458,0.672795,0.456134,0.665079,0.666634,0.675585


In [76]:
gdump(X_train, 'X_train_all_20d')

In [77]:
del X_train, tmp

# 训练

## 取出数据

In [78]:
X_train_all = read_param('X_train_all')
y_train_all = gload('y_train_all')
X_train_all_20d = gload('X_train_all_20d')

# 拼接数据
X_train_all = pd.concat([X_train_all, X_train_all_20d], axis=1)
del X_train_all_20d
X_train_all

,freq,gender,hour,q_ans_count,q_diff_qa_days_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_min,q_diff_qa_days_std,q_diff_qa_days_sum,...,aaa,bbb,ccc,ddd,eee,v6_1,v6_2,v6_3,v6_123,"v6_1,2,3"
0,5,3,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.000000,0.000000,0.000000,0.330838,0.456134,0.328747,0.329541,0.329874
1,5,3,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.750000,0.9,0.818182,0.322034,0.082609,0.499043,0.517830,0.139592,0.139055,0.125604
2,2,3,4,32.0,32.0,13.0,9.531250,0.0,4.634961,305.0,...,0.000000,0.0,0.000000,0.000000,0.000000,0.330838,0.456134,0.328747,0.329541,0.329874
3,1,2,20,3.0,3.0,234.0,222.333328,215.0,10.214369,667.0,...,1.000000,1.0,1.000000,0.508475,0.049180,0.562440,0.456134,0.518578,0.530327,0.575211
4,5,1,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.583333,1.0,0.583333,0.169492,0.003221,0.694262,0.569675,0.622583,0.668858,0.696415
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2593664,1,1,20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.000000,0.0,0.000000,0.000000,0.000000,0.330838,0.456134,0.328747,0.329541,0.329874
2593665,1,3,15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.666667,1.0,0.666667,0.237288,0.028000,0.517593,0.456134,0.511814,0.555835,0.524057
2593666,1,3,18,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.583333,1.0,0.583333,0.254237,0.030000,0.541721,0.964831,0.521709,0.687276,0.963946
2593667,1,2,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.250000,1.0,0.250000,0.050847,0.000458,0.672795,0.456134,0.665079,0.666634,0.675585


In [82]:
X_train_all.rename(columns={'v6_1,2,3':'v6_1_2_3'}, inplace=True)

In [17]:
# # 取出过一次后再取就不用处理啦
# X_train_all = gload('X_trian_v5_all_reduced')
# y_train_all = gload('y_train_all')

Compress data.（超强压缩算法）

In [22]:
# # @from: https://www.kaggle.com/arjanso/reducing-dataframe-memory-size-by-65/code
# # @liscense: Apache 2.0
# # @author: weijian
# def reduce_mem_usage(props):
#     # 计算当前内存
#     start_mem_usg = props.memory_usage().sum() / 1024 ** 2
#     print("Memory usage of the dataframe is :", start_mem_usg, "MB")
    
#     # 哪些列包含空值，空值用-999填充。why：因为np.nan当做float处理
#     NAlist = []
#     for col in props.columns:
#         # 这里只过滤了objectd格式，如果你的代码中还包含其他类型，请一并过滤
#         if (props[col].dtypes != object):
            
#             print("**************************")
#             print("columns: ", col)
#             print("dtype before", props[col].dtype)
            
#             # 判断是否是int类型
#             isInt = False
#             mmax = props[col].max()
#             mmin = props[col].min()
            
#             # Integer does not support NA, therefore Na needs to be filled
#             if not np.isfinite(props[col]).all():
#                 NAlist.append(col)
#                 props[col].fillna(-999, inplace=True) # 用-999填充
                
#             # test if column can be converted to an integer
#             asint = props[col].fillna(0).astype(np.int64)
#             result = np.fabs(props[col] - asint)
#             result = result.sum()
#             if result < 0.01: # 绝对误差和小于0.01认为可以转换的，要根据task修改
#                 isInt = True
            
#             # make interger / unsigned Integer datatypes
#             if isInt:
#                 if mmin >= 0: # 最小值大于0，转换成无符号整型
#                     if mmax <= 255:
#                         props[col] = props[col].astype(np.uint8)
#                     elif mmax <= 65535:
#                         props[col] = props[col].astype(np.uint16)
#                     elif mmax <= 4294967295:
#                         props[col] = props[col].astype(np.uint32)
#                     else:
#                         props[col] = props[col].astype(np.uint64)
#                 else: # 转换成有符号整型
#                     if mmin > np.iinfo(np.int8).min and mmax < np.iinfo(np.int8).max:
#                         props[col] = props[col].astype(np.int8)
#                     elif mmin > np.iinfo(np.int16).min and mmax < np.iinfo(np.int16).max:
#                         props[col] = props[col].astype(np.int16)
#                     elif mmin > np.iinfo(np.int32).min and mmax < np.iinfo(np.int32).max:
#                         props[col] = props[col].astype(np.int32)
#                     elif mmin > np.iinfo(np.int64).min and mmax < np.iinfo(np.int64).max:
#                         props[col] = props[col].astype(np.int64)  
#             else: # 注意：这里对于float都转换成float16，需要根据你的情况自己更改
#                 props[col] = props[col].astype(np.float32)
            
#             print("dtype after", props[col].dtype)
#             print("********************************")
#     print("___MEMORY USAGE AFTER COMPLETION:___")
#     mem_usg = props.memory_usage().sum() / 1024**2 
#     print("Memory usage is: ",mem_usg," MB")
#     print("This is ",100*mem_usg/start_mem_usg,"% of the initial size")
#     return props, NAlist

In [50]:
# gdump(X_train_all, 'X_trian_v5_all_reduced')

## LGBM分类器训练

### cv交叉验证

In [26]:
# 221 input
X_train_all = read_param('X_train_all')
y_train_all = gload('y_train_all')
feature_cols = gload('feature_cols')
feature_cols.append('colv5')
print('X_train_all.shape:', X_train_all.shape)

X_tmp, y_tmp = X_train_all[int(0.2*len(X_train_all)):], y_train_all[int(0.2*len(X_train_all)):]
train_data = lgb.Dataset(X_tmp[feature_cols], y_tmp)
del X_train_all, y_train_all

model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.3,
                           n_estimators=200, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
                           seed=666, 
                           silent=True,
                           device='gpu',
                          )

cv_res = lgb.cv(model_lgb.get_params(), train_data, num_boost_round=5,
                nfold=5, metrics=['logloss', 'auc'], verbose_eval=5,
                early_stopping_rounds=10, stratified=False, shuffle=True,)

print('best n_estimators:', len(cv_res['auc-mean']))
print('best cv score:', cv_res['auc-mean'][-1])
del X_tmp, y_tmp, train_data

[5]	cv_agg's auc: 0.865387 + 0.00112399
[10]	cv_agg's auc: 0.869967 + 0.00107055
[15]	cv_agg's auc: 0.874106 + 0.000953568
[20]	cv_agg's auc: 0.876051 + 0.00102926
[25]	cv_agg's auc: 0.87731 + 0.00103074
[30]	cv_agg's auc: 0.878381 + 0.000978689
[35]	cv_agg's auc: 0.878787 + 0.000996533
[40]	cv_agg's auc: 0.879695 + 0.000988646
[45]	cv_agg's auc: 0.880335 + 0.000954589
[50]	cv_agg's auc: 0.880247 + 0.000953845
[55]	cv_agg's auc: 0.880491 + 0.000934943
[60]	cv_agg's auc: 0.881237 + 0.000896446
[65]	cv_agg's auc: 0.881617 + 0.000840497
[70]	cv_agg's auc: 0.882076 + 0.000803534
[75]	cv_agg's auc: 0.882436 + 0.000809805
[80]	cv_agg's auc: 0.882754 + 0.000792338
[85]	cv_agg's auc: 0.883035 + 0.000822719
[90]	cv_agg's auc: 0.883287 + 0.000864668
[95]	cv_agg's auc: 0.883404 + 0.000848585
[100]	cv_agg's auc: 0.883607 + 0.000828861
[105]	cv_agg's auc: 0.883681 + 0.000831901
[110]	cv_agg's auc: 0.883782 + 0.000836735
[115]	cv_agg's auc: 0.883881 + 0.000820045
[120]	cv_agg's auc: 0.88395 + 0.0008

In [48]:
# 230 input
X_train_all = read_param('X_train_all')
X_train_all_v5 = gload('X_train_all_v5')
y_train_all = gload('y_train_all')
feature_cols = gload('feature_cols')
feature_cols_v5 = gload('feature_cols_v5')

X_train_all = pd.concat([X_train_all, X_train_all_v5[feature_cols_v5]], axis=1)
feature_cols.extend(feature_cols_v5)
print('X_train_all.shape:', X_train_all.shape)
del X_train_all_v5, feature_cols_v5

X_tmp, y_tmp = X_train_all[int(0.2*len(X_train_all)):], y_train_all[int(0.2*len(X_train_all)):]
train_data = lgb.Dataset(X_tmp, y_tmp)
del X_train_all, y_train_all

model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.3,
                           n_estimators=200, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
                           seed=666, 
                           silent=True,
                           device='gpu',
                          )

cv_res = lgb.cv(model_lgb.get_params(), train_data, num_boost_round=5,
                nfold=5, metrics=['logloss', 'auc'], verbose_eval=5,
                early_stopping_rounds=10, stratified=False, shuffle=True,)

print('best n_estimators:', len(cv_res['auc-mean']))
print('best cv score:', cv_res['auc-mean'][-1])
del X_tmp, y_tmp, train_data

[5]	cv_agg's auc: 0.864006 + 0.00107252
[10]	cv_agg's auc: 0.870316 + 0.0011868
[15]	cv_agg's auc: 0.876153 + 0.00095644
[20]	cv_agg's auc: 0.878644 + 0.000949346
[25]	cv_agg's auc: 0.880628 + 0.00088579
[30]	cv_agg's auc: 0.881883 + 0.000844579
[35]	cv_agg's auc: 0.882549 + 0.000795606
[40]	cv_agg's auc: 0.883765 + 0.00078133
[45]	cv_agg's auc: 0.884832 + 0.000801308
[50]	cv_agg's auc: 0.884714 + 0.000812119
[55]	cv_agg's auc: 0.885051 + 0.000931006
[60]	cv_agg's auc: 0.88604 + 0.000836551
[65]	cv_agg's auc: 0.886661 + 0.000752837
[70]	cv_agg's auc: 0.887247 + 0.000701844
[75]	cv_agg's auc: 0.887777 + 0.000717053
[80]	cv_agg's auc: 0.888298 + 0.000720959
[85]	cv_agg's auc: 0.888662 + 0.000723602
[90]	cv_agg's auc: 0.88903 + 0.000734121
[95]	cv_agg's auc: 0.88912 + 0.000723676
[100]	cv_agg's auc: 0.889406 + 0.000693491
[105]	cv_agg's auc: 0.889534 + 0.00073621
[110]	cv_agg's auc: 0.889704 + 0.00075198
[115]	cv_agg's auc: 0.889847 + 0.000714147
[120]	cv_agg's auc: 0.889894 + 0.000712545

In [49]:
# 231 input
X_train_all = read_param('X_train_all')
X_train_all_v5 = gload('X_train_all_v5')
y_train_all = gload('y_train_all')
feature_cols = gload('feature_cols')
feature_cols_v5 = gload('feature_cols_v5')
colv5 = gload('colv5')

X_train_all = pd.concat([X_train_all, X_train_all_v5[feature_cols_v5]], axis=1)
X_train_all = pd.concat([X_train_all, colv5['colv5']], axis=1)
feature_cols.extend(feature_cols_v5)
print('X_train_all.shape:', X_train_all.shape)
del X_train_all_v5, feature_cols_v5, colv5

X_tmp, y_tmp = X_train_all[int(0.2*len(X_train_all)):], y_train_all[int(0.2*len(X_train_all)):]
train_data = lgb.Dataset(X_tmp[feature_cols], y_tmp)
del X_train_all, y_train_all

model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.3,
                           n_estimators=200, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
                           seed=666, 
                           silent=True,
                           device='gpu',
                          )

cv_res = lgb.cv(model_lgb.get_params(), train_data, num_boost_round=5,
                nfold=5, metrics=['logloss', 'auc'], verbose_eval=5,
                early_stopping_rounds=10, stratified=False, shuffle=True,)

print('best n_estimators:', len(cv_res['auc-mean']))
print('best cv score:', cv_res['auc-mean'][-1])
del X_tmp, y_tmp, train_data

X_train_all.shape: (2593669, 231)
[5]	cv_agg's auc: 0.866971 + 0.000909077
[10]	cv_agg's auc: 0.87246 + 0.000856691
[15]	cv_agg's auc: 0.877834 + 0.000777032
[20]	cv_agg's auc: 0.880093 + 0.000790816
[25]	cv_agg's auc: 0.881775 + 0.000788219
[30]	cv_agg's auc: 0.883058 + 0.000791931
[35]	cv_agg's auc: 0.88354 + 0.000874643
[40]	cv_agg's auc: 0.884897 + 0.000980263
[45]	cv_agg's auc: 0.885874 + 0.000975463
[50]	cv_agg's auc: 0.885703 + 0.000943437
[55]	cv_agg's auc: 0.885944 + 0.000953915
[60]	cv_agg's auc: 0.886889 + 0.000866099
[65]	cv_agg's auc: 0.887457 + 0.000829388
[70]	cv_agg's auc: 0.888021 + 0.000874698
[75]	cv_agg's auc: 0.888495 + 0.000861532
[80]	cv_agg's auc: 0.888985 + 0.000867859
[85]	cv_agg's auc: 0.889338 + 0.000837581
[90]	cv_agg's auc: 0.889612 + 0.000772813
[95]	cv_agg's auc: 0.889727 + 0.000797705
[100]	cv_agg's auc: 0.889909 + 0.000760363
[105]	cv_agg's auc: 0.890038 + 0.000816991
[110]	cv_agg's auc: 0.890129 + 0.000803543
[115]	cv_agg's auc: 0.890246 + 0.000773463

In [84]:
# 240 input
# X_train_all = read_param('X_train_all')
# y_train_all = gload('y_train_all')
# X_train_all_20d = gload('X_train_all_20d')

# # 拼接数据
# X_train_all = pd.concat([X_train_all, X_train_all_20d], axis=1)
# del X_train_all_20d
print('X_train_all.shape:', X_train_all.shape)

X_tmp, y_tmp = X_train_all[int(0.2*len(X_train_all)):], y_train_all[int(0.2*len(X_train_all)):]
train_data = lgb.Dataset(X_tmp, y_tmp)
# del X_train_all, y_train_all

model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.3,
                           n_estimators=200, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
                           seed=666, 
                           silent=True,
                           device='gpu',
                          )

cv_res = lgb.cv(model_lgb.get_params(), train_data, num_boost_round=5,
                nfold=5, metrics=['logloss', 'auc'], verbose_eval=5,
                early_stopping_rounds=10, stratified=False, shuffle=True,)

print('best n_estimators:', len(cv_res['auc-mean']))
print('best cv score:', cv_res['auc-mean'][-1])
del X_tmp, y_tmp, train_data

X_train_all.shape: (2593669, 240)
[5]	cv_agg's auc: 0.87977 + 0.000210668
[10]	cv_agg's auc: 0.885055 + 0.000460811
[15]	cv_agg's auc: 0.890019 + 0.000483397
[20]	cv_agg's auc: 0.892605 + 0.000499653
[25]	cv_agg's auc: 0.894218 + 0.000504849
[30]	cv_agg's auc: 0.895412 + 0.000525371
[35]	cv_agg's auc: 0.895894 + 0.000533402
[40]	cv_agg's auc: 0.897116 + 0.000527951
[45]	cv_agg's auc: 0.89809 + 0.000539416
[50]	cv_agg's auc: 0.897942 + 0.000544829
[55]	cv_agg's auc: 0.898012 + 0.000521943
[60]	cv_agg's auc: 0.899025 + 0.000461869
[65]	cv_agg's auc: 0.899613 + 0.000497419
[70]	cv_agg's auc: 0.900157 + 0.000478274
[75]	cv_agg's auc: 0.900597 + 0.000509573
[80]	cv_agg's auc: 0.901057 + 0.000456521
[85]	cv_agg's auc: 0.901426 + 0.000462399
[90]	cv_agg's auc: 0.901737 + 0.000454403
[95]	cv_agg's auc: 0.901844 + 0.000468914
[100]	cv_agg's auc: 0.902094 + 0.000442695
[105]	cv_agg's auc: 0.902205 + 0.000436855
[110]	cv_agg's auc: 0.902328 + 0.000435657
[115]	cv_agg's auc: 0.90244 + 0.000424058


In [86]:
gdump(X_train_all_20d, 'X_train_all_20d')

### train

In [5]:
# 240 input
X_train_all = read_param('X_train_all')
X_train_all_20d = gload('X_train_all_20d')
y_train_all = gload('y_train_all')

X_train_all = pd.concat([X_train_all, X_train_all_20d], axis=1)
print('X_train_all.shape:', X_train_all.shape)
del X_train_all_20d

X_train_all.shape: (2593669, 240)


In [6]:
X_val = X_train_all[:50000]
y_val = y_train_all[:50000]

In [7]:
model_lgb = LGBMClassifier(boosting_type='dart',
                           objective='binary', 
                           learning_rate=0.005,
                           n_estimators=10000, 
                           max_depth=20,
                           num_leaves=190,
                           min_child_samples=210,
                           min_child_weight=0.055,
                           reg_alpha=0.005,
                           reg_lambda=21,
                           n_jobs=16,
#                            seed=666, 
                           silent=True,
                           device='gpu',)
model_lgb.fit(X_train_all, y_train_all,
              eval_metric=['logloss', 'auc'],
              eval_set=[(X_val, y_val)],
              early_stopping_rounds=50)

[1]	valid_0's auc: 0.860044	valid_0's binary_logloss: 0.437343
[2]	valid_0's auc: 0.861692	valid_0's binary_logloss: 0.435778
[3]	valid_0's auc: 0.861866	valid_0's binary_logloss: 0.43425
[4]	valid_0's auc: 0.861862	valid_0's binary_logloss: 0.432751
[5]	valid_0's auc: 0.862033	valid_0's binary_logloss: 0.431283
[6]	valid_0's auc: 0.862166	valid_0's binary_logloss: 0.429841
[7]	valid_0's auc: 0.862702	valid_0's binary_logloss: 0.42842
[8]	valid_0's auc: 0.862694	valid_0's binary_logloss: 0.428929
[9]	valid_0's auc: 0.862944	valid_0's binary_logloss: 0.427525
[10]	valid_0's auc: 0.863092	valid_0's binary_logloss: 0.426149
[11]	valid_0's auc: 0.86316	valid_0's binary_logloss: 0.4248
[12]	valid_0's auc: 0.863165	valid_0's binary_logloss: 0.42527
[13]	valid_0's auc: 0.863109	valid_0's binary_logloss: 0.423931
[14]	valid_0's auc: 0.863371	valid_0's binary_logloss: 0.42262
[15]	valid_0's auc: 0.863841	valid_0's binary_logloss: 0.421323
[16]	valid_0's auc: 0.864098	valid_0's binary_logloss: 0

[130]	valid_0's auc: 0.871634	valid_0's binary_logloss: 0.440059
[131]	valid_0's auc: 0.871638	valid_0's binary_logloss: 0.440484
[132]	valid_0's auc: 0.871609	valid_0's binary_logloss: 0.439452
[133]	valid_0's auc: 0.871633	valid_0's binary_logloss: 0.438419
[134]	valid_0's auc: 0.87165	valid_0's binary_logloss: 0.445999
[135]	valid_0's auc: 0.871668	valid_0's binary_logloss: 0.444923
[136]	valid_0's auc: 0.871706	valid_0's binary_logloss: 0.455306
[137]	valid_0's auc: 0.871716	valid_0's binary_logloss: 0.455776
[138]	valid_0's auc: 0.871745	valid_0's binary_logloss: 0.454631
[139]	valid_0's auc: 0.87174	valid_0's binary_logloss: 0.453494
[140]	valid_0's auc: 0.871738	valid_0's binary_logloss: 0.454082
[141]	valid_0's auc: 0.871738	valid_0's binary_logloss: 0.454541
[142]	valid_0's auc: 0.871726	valid_0's binary_logloss: 0.45503
[143]	valid_0's auc: 0.871735	valid_0's binary_logloss: 0.45389
[144]	valid_0's auc: 0.871735	valid_0's binary_logloss: 0.454301
[145]	valid_0's auc: 0.871768

[257]	valid_0's auc: 0.872949	valid_0's binary_logloss: 0.441738
[258]	valid_0's auc: 0.873112	valid_0's binary_logloss: 0.440692
[259]	valid_0's auc: 0.873121	valid_0's binary_logloss: 0.44127
[260]	valid_0's auc: 0.873119	valid_0's binary_logloss: 0.441907
[261]	valid_0's auc: 0.873121	valid_0's binary_logloss: 0.442473
[262]	valid_0's auc: 0.873196	valid_0's binary_logloss: 0.441422
[263]	valid_0's auc: 0.873198	valid_0's binary_logloss: 0.440382
[264]	valid_0's auc: 0.873281	valid_0's binary_logloss: 0.439344
[265]	valid_0's auc: 0.873345	valid_0's binary_logloss: 0.438315
[266]	valid_0's auc: 0.873413	valid_0's binary_logloss: 0.437293
[267]	valid_0's auc: 0.873406	valid_0's binary_logloss: 0.437865
[268]	valid_0's auc: 0.873482	valid_0's binary_logloss: 0.436846
[269]	valid_0's auc: 0.873487	valid_0's binary_logloss: 0.437425
[270]	valid_0's auc: 0.87357	valid_0's binary_logloss: 0.436404
[271]	valid_0's auc: 0.873569	valid_0's binary_logloss: 0.436993
[272]	valid_0's auc: 0.8736

[384]	valid_0's auc: 0.875458	valid_0's binary_logloss: 0.418079
[385]	valid_0's auc: 0.875451	valid_0's binary_logloss: 0.418645
[386]	valid_0's auc: 0.875484	valid_0's binary_logloss: 0.417755
[387]	valid_0's auc: 0.875504	valid_0's binary_logloss: 0.416874
[388]	valid_0's auc: 0.875546	valid_0's binary_logloss: 0.415998
[389]	valid_0's auc: 0.87558	valid_0's binary_logloss: 0.41513
[390]	valid_0's auc: 0.875603	valid_0's binary_logloss: 0.414268
[391]	valid_0's auc: 0.875598	valid_0's binary_logloss: 0.414806
[392]	valid_0's auc: 0.875617	valid_0's binary_logloss: 0.413946
[393]	valid_0's auc: 0.875638	valid_0's binary_logloss: 0.41309
[394]	valid_0's auc: 0.875647	valid_0's binary_logloss: 0.412244
[395]	valid_0's auc: 0.875685	valid_0's binary_logloss: 0.411399
[396]	valid_0's auc: 0.875682	valid_0's binary_logloss: 0.411899
[397]	valid_0's auc: 0.875701	valid_0's binary_logloss: 0.411057
[398]	valid_0's auc: 0.87574	valid_0's binary_logloss: 0.410224
[399]	valid_0's auc: 0.875764

[511]	valid_0's auc: 0.87805	valid_0's binary_logloss: 0.383791
[512]	valid_0's auc: 0.878072	valid_0's binary_logloss: 0.383137
[513]	valid_0's auc: 0.878094	valid_0's binary_logloss: 0.382493
[514]	valid_0's auc: 0.878117	valid_0's binary_logloss: 0.381852
[515]	valid_0's auc: 0.878138	valid_0's binary_logloss: 0.381214
[516]	valid_0's auc: 0.878137	valid_0's binary_logloss: 0.381612
[517]	valid_0's auc: 0.878137	valid_0's binary_logloss: 0.382017
[518]	valid_0's auc: 0.878139	valid_0's binary_logloss: 0.382354
[519]	valid_0's auc: 0.878138	valid_0's binary_logloss: 0.382771
[520]	valid_0's auc: 0.878167	valid_0's binary_logloss: 0.382121
[521]	valid_0's auc: 0.878188	valid_0's binary_logloss: 0.38148
[522]	valid_0's auc: 0.878186	valid_0's binary_logloss: 0.381899
[523]	valid_0's auc: 0.878179	valid_0's binary_logloss: 0.382319
[524]	valid_0's auc: 0.878176	valid_0's binary_logloss: 0.382745
[525]	valid_0's auc: 0.878175	valid_0's binary_logloss: 0.383138
[526]	valid_0's auc: 0.8782

...

[9519]	valid_0's auc: 0.913092	valid_0's binary_logloss: 0.250131
[9520]	valid_0's auc: 0.913092	valid_0's binary_logloss: 0.250126
[9521]	valid_0's auc: 0.913093	valid_0's binary_logloss: 0.250128
[9522]	valid_0's auc: 0.913094	valid_0's binary_logloss: 0.250131
[9523]	valid_0's auc: 0.913097	valid_0's binary_logloss: 0.250124
[9524]	valid_0's auc: 0.913098	valid_0's binary_logloss: 0.250127
[9525]	valid_0's auc: 0.913099	valid_0's binary_logloss: 0.25013
[9526]	valid_0's auc: 0.913101	valid_0's binary_logloss: 0.250122
[9527]	valid_0's auc: 0.913103	valid_0's binary_logloss: 0.250126
[9528]	valid_0's auc: 0.913104	valid_0's binary_logloss: 0.250129
[9529]	valid_0's auc: 0.913105	valid_0's binary_logloss: 0.250123
[9530]	valid_0's auc: 0.913107	valid_0's binary_logloss: 0.250116
[9531]	valid_0's auc: 0.91311	valid_0's binary_logloss: 0.250107
[9532]	valid_0's auc: 0.913113	valid_0's binary_logloss: 0.250101
[9533]	valid_0's auc: 0.913114	valid_0's binary_logloss: 0.250103
[9534]	valid

[9644]	valid_0's auc: 0.913281	valid_0's binary_logloss: 0.249867
[9645]	valid_0's auc: 0.913282	valid_0's binary_logloss: 0.249869
[9646]	valid_0's auc: 0.913283	valid_0's binary_logloss: 0.249864
[9647]	valid_0's auc: 0.913285	valid_0's binary_logloss: 0.249858
[9648]	valid_0's auc: 0.913286	valid_0's binary_logloss: 0.249859
[9649]	valid_0's auc: 0.913286	valid_0's binary_logloss: 0.249853
[9650]	valid_0's auc: 0.913287	valid_0's binary_logloss: 0.249856
[9651]	valid_0's auc: 0.913288	valid_0's binary_logloss: 0.24986
[9652]	valid_0's auc: 0.913289	valid_0's binary_logloss: 0.249862
[9653]	valid_0's auc: 0.913291	valid_0's binary_logloss: 0.249856
[9654]	valid_0's auc: 0.913293	valid_0's binary_logloss: 0.249849
[9655]	valid_0's auc: 0.913293	valid_0's binary_logloss: 0.249851
[9656]	valid_0's auc: 0.913295	valid_0's binary_logloss: 0.249845
[9657]	valid_0's auc: 0.913297	valid_0's binary_logloss: 0.249848
[9658]	valid_0's auc: 0.913298	valid_0's binary_logloss: 0.249842
[9659]	vali

[9769]	valid_0's auc: 0.913479	valid_0's binary_logloss: 0.249584
[9770]	valid_0's auc: 0.913482	valid_0's binary_logloss: 0.249577
[9771]	valid_0's auc: 0.913483	valid_0's binary_logloss: 0.249583
[9772]	valid_0's auc: 0.913484	valid_0's binary_logloss: 0.249585
[9773]	valid_0's auc: 0.913485	valid_0's binary_logloss: 0.249588
[9774]	valid_0's auc: 0.913486	valid_0's binary_logloss: 0.24959
[9775]	valid_0's auc: 0.913487	valid_0's binary_logloss: 0.249593
[9776]	valid_0's auc: 0.913488	valid_0's binary_logloss: 0.249595
[9777]	valid_0's auc: 0.91349	valid_0's binary_logloss: 0.249589
[9778]	valid_0's auc: 0.913492	valid_0's binary_logloss: 0.249582
[9779]	valid_0's auc: 0.913496	valid_0's binary_logloss: 0.249575
[9780]	valid_0's auc: 0.913498	valid_0's binary_logloss: 0.249569
[9781]	valid_0's auc: 0.913501	valid_0's binary_logloss: 0.249562
[9782]	valid_0's auc: 0.913503	valid_0's binary_logloss: 0.249557
[9783]	valid_0's auc: 0.913504	valid_0's binary_logloss: 0.249561
[9784]	valid

[9894]	valid_0's auc: 0.913679	valid_0's binary_logloss: 0.249329
[9895]	valid_0's auc: 0.913682	valid_0's binary_logloss: 0.249322
[9896]	valid_0's auc: 0.913682	valid_0's binary_logloss: 0.249323
[9897]	valid_0's auc: 0.913684	valid_0's binary_logloss: 0.249316
[9898]	valid_0's auc: 0.913686	valid_0's binary_logloss: 0.249309
[9899]	valid_0's auc: 0.913688	valid_0's binary_logloss: 0.249312
[9900]	valid_0's auc: 0.913691	valid_0's binary_logloss: 0.249305
[9901]	valid_0's auc: 0.913694	valid_0's binary_logloss: 0.249299
[9902]	valid_0's auc: 0.913698	valid_0's binary_logloss: 0.249292
[9903]	valid_0's auc: 0.9137	valid_0's binary_logloss: 0.249285
[9904]	valid_0's auc: 0.913701	valid_0's binary_logloss: 0.249281
[9905]	valid_0's auc: 0.913701	valid_0's binary_logloss: 0.249283
[9906]	valid_0's auc: 0.913702	valid_0's binary_logloss: 0.249286
[9907]	valid_0's auc: 0.913706	valid_0's binary_logloss: 0.249278
[9908]	valid_0's auc: 0.913708	valid_0's binary_logloss: 0.249272
[9909]	valid

LGBMClassifier(boosting_type='dart', class_weight=None, colsample_bytree=1.0,
               device='gpu', importance_type='split', learning_rate=0.005,
               max_depth=20, min_child_samples=210, min_child_weight=0.055,
               min_split_gain=0.0, n_estimators=10000, n_jobs=16,
               num_leaves=190, objective='binary', random_state=None,
               reg_alpha=0.005, reg_lambda=21, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [8]:
gdump(model_lgb, 'model_lgb_240')

In [55]:
del X_train_all, y_train_all

## 集成学习Ensemble

In [2]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Activation, BatchNormalization, Dense, Dropout, Flatten

import os
import tensorflow as tf
from keras.backend import set_session
os.environ["CUDA_VISIBLE_DEVICES"] = "3"
config = tf.compat.v1.ConfigProto()
# config.gpu_options.per_process_gpu_memory_fraction = 0.8  # 占用GPU百分比
config.gpu_options.allow_growth = True
set_session(tf.compat.v1.Session(config=config)) # 此处不同

BATCH_SIZE = 8192
EPOCHS = 5
LR = 0.01

callbacks=[
    tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', 
                                         factor=0.1, 
                                         patience=3, 
                                         verbose=1, 
                                         mode='auto', 
                                         min_delta=0.0001, 
                                         cooldown=0, 
                                         min_lr=0),  # 当指标变化小时，减少学习率
]

Using TensorFlow backend.


In [5]:
def get_easy_net(input_dim, optimizer='rmsprop', loss='binary_crossentropy'):
    m = Sequential()

    m.add(Dense(256, activation='tanh', input_shape=(input_dim,)))
    m.add(Dropout(0.3))
    m.add(Dense(512, activation='relu'))
    m.add(Dropout(0.3))
    m.add(Dense(512, activation='tanh'))
    m.add(Dropout(0.3))
    m.add(Dense(256, activation='linear'))
    m.add(Dense(1, activation='sigmoid'))

    m.compile(optimizer=optimizer,
              loss=loss,
              metrics=['accuracy'])

    return m

In [3]:
X_train_all = read_param('X_train_all')
X_train_all_20d = gload('X_train_all_20d')
y_train_all = gload('y_train_all')
model_lgb_v4 = gload('model_lgb_v4')

In [4]:
X_train = pd.DataFrame()
X_train['v4'] = model_lgb_v4.predict_proba(X_train_all)[:, 1]
X_train = pd.concat([X_train, X_train_all_20d[['v6_1', 'v6_2', 'v6_3', 'v6_123', 'v6_1_2_3']]], axis=1)
y_train = y_train_all
del X_train_all, y_train_all, X_train_all_20d
X_train

,v4,v6_1,v6_2,v6_3,v6_123,v6_1_2_3
0,0.006213,0.330838,0.456134,0.328747,0.329541,0.329874
1,0.077625,0.499043,0.517830,0.139592,0.139055,0.125604
2,0.093296,0.330838,0.456134,0.328747,0.329541,0.329874
3,0.083525,0.562440,0.456134,0.518578,0.530327,0.575211
4,0.439985,0.694262,0.569675,0.622583,0.668858,0.696415
...,...,...,...,...,...,...
2593664,0.012299,0.330838,0.456134,0.328747,0.329541,0.329874
2593665,0.071902,0.517593,0.456134,0.511814,0.555835,0.524057
2593666,0.190918,0.541721,0.964831,0.521709,0.687276,0.963946
2593667,0.272731,0.672795,0.456134,0.665079,0.666634,0.675585


In [6]:
# 过采样
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

logging.info('y_train distribution is %s' % Counter(y_train))
ros = RandomOverSampler(random_state=0)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
logging.info('y_resampled distribution is %s' % Counter(y_resampled))

# del X_train, y_train

[2019-12-16 21:15:17,384] INFO: y_train distribution is Counter({0.0: 2178656, 1.0: 415013})
[2019-12-16 21:15:20,184] INFO: y_resampled distribution is Counter({0.0: 2178656, 1.0: 2178656})


In [7]:
# 手动shuffle
X_resampled = X_resampled.values
y_resampled = y_resampled.values
np.random.seed(100)
np.random.shuffle(X_resampled)
np.random.seed(100)
np.random.shuffle(y_resampled)

In [8]:
model = get_easy_net(input_dim=6)

[2019-12-16 21:15:28,939] WARNING: From /home/zengrui/miniconda3/lib/python3.7/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
H = model.fit(X_resampled, y_resampled, 
              epochs=100, batch_size=BATCH_SIZE, #validation_split=0.1,
              shuffle='batch', callbacks=callbacks, verbose=1)

[2019-12-16 21:15:53,535] WARNING: From /home/zengrui/miniconda3/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/100
4357312/4357312 [==============================] - 9s 2us/step - loss: 0.4338 - accuracy: 0.7956
Epoch 2/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.4142 - accuracy: 0.8054
Epoch 3/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.4124 - accuracy: 0.8058
Epoch 4/100
4357312/4357312 [==============================] - 8s 2us/step - loss: 0.4114 - accuracy: 0.8059
Epoch 5/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.4109 - accuracy: 0.8061
Epoch 6/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.4105 - accuracy: 0.8063
Epoch 7/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.4100 - accuracy: 0.8063
Epoch 8/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.4097 - accuracy: 0.8064
Epoch 9/100
4357312/4357312 [==============================] - 7s 2us/step - loss: 0.4094 - accuracy: 0.8064
Epoch 10/100
435731

In [10]:
gdump(model, 'model_nn_ensemble_v6')

In [117]:
model = gload('model_ensemble_v6')

In [11]:
del X_resampled, y_resampled, X_train, y_train

# Result

## predict 240

In [18]:
# load data
test = read_param('test')
sub = read_param('sub')
feature_cols = gload('feature_cols')
feature_cols_v5 = gload('feature_cols_v5')
test_v6 = gload('test_v6_2')
del test_v6['index']

feature_cols = gload('feature_cols')
feature_cols_v6 = ['a', 'b', 'c', 'd', 'e', 'aa', 'bb', 'cc', 'dd', 'ee', 'aaa', 'bbb', 'ccc', 'ddd', 'eee']
feature_cols.extend(feature_cols_v6)
feature_cols.extend(['v6_1', 'v6_2', 'v6_3', 'v6_123', 'v6_1_2_3'])
len(feature_cols)

240

In [20]:
from sklearn.preprocessing import MinMaxScaler

mms = MinMaxScaler()
x = test_v6[feature_cols_v6].values
x = mms.fit_transform(x)
x = pd.DataFrame(x)
x.columns = feature_cols_v6
x[:5]

,a,b,c,d,e,aa,bb,cc,dd,ee,aaa,bbb,ccc,ddd,eee
0,0.750000,1.000000,0.75,0.114679,0.001163,1.0,1.0,1.0,0.041849,0.00008,0.416667,1.000000,0.416667,0.113636,0.001147
1,0.750000,1.000000,0.75,0.091743,0.015733,0.0,0.0,0.0,0.000000,0.00000,0.666667,1.000000,0.800000,0.522727,0.106977
2,0.666667,0.800000,1.00,0.082569,0.047059,0.0,0.0,0.0,0.000000,0.00000,0.250000,1.000000,0.600000,0.068182,0.024000
3,0.833333,0.833333,1.00,0.178899,0.099048,0.0,0.0,0.0,0.000000,0.00000,0.583333,1.000000,0.700000,0.295455,0.185714
4,0.750000,0.750000,1.00,0.155963,0.116239,0.0,0.0,0.0,0.000000,0.00000,0.500000,0.857143,0.666667,0.181818,0.061538


In [21]:
model_nn_col1 = gload('model_nn_col1')
model_nn_col2 = gload('model_nn_col2')
model_nn_col3 = gload('model_nn_col3')
model_nn_col123 = gload('model_nn_col123')
model_nn_col1_2_3 = gload('model_nn_col1_2_3')

c1 = model_nn_col1.predict(x.iloc[:, 0:5], batch_size=8192)
c2 = model_nn_col2.predict(x.iloc[:, 5:10], batch_size=8192)
c3 = model_nn_col3.predict(x.iloc[:, 10:15], batch_size=8192)
c4 = model_nn_col123.predict(x.iloc[:, 0:15], batch_size=8192)
c5 = model_nn_col1_2_3.predict(np.hstack([c1, c2, c3]), batch_size=8192)
c = np.hstack([c1, c2, c3, c4, c5])

c = pd.DataFrame(c)
c.columns = ['v6_1', 'v6_2', 'v6_3', 'v6_123', 'v6_1_2_3']
c

,v6_1,v6_2,v6_3,v6_123,v6_1_2_3
0,0.653088,0.650794,0.692645,0.680957,0.662520
1,0.502849,0.456134,0.810591,0.765378,0.812050
2,0.243186,0.456134,0.615402,0.196424,0.331868
3,0.158300,0.456134,0.198938,0.224858,0.174435
4,0.182655,0.456134,0.161311,0.192835,0.167960
...,...,...,...,...,...
1141713,0.330838,0.456134,0.328747,0.329541,0.329874
1141714,0.130783,0.456134,0.650737,0.123888,0.142459
1141715,0.158856,0.456134,0.167935,0.127028,0.152819
1141716,0.330838,0.456134,0.328747,0.329541,0.329874


In [22]:
test_240 = pd.concat([test, test_v6, c], axis=1)
test_240

,day,freq,gender,hour,label,q_ans_count,q_diff_qa_days_count,q_diff_qa_days_max,q_diff_qa_days_mean,q_diff_qa_days_min,...,aaa,bbb,ccc,ddd,eee,v6_1,v6_2,v6_3,v6_123,v6_1_2_3
0,3871,1,3,6,NaN,NaN,NaN,NaN,NaN,NaN,...,5,1.000000,0.416667,5,0.005734,0.653088,0.650794,0.692645,0.680957,0.662520
1,3871,1,2,23,NaN,NaN,NaN,NaN,NaN,NaN,...,8,1.000000,0.800000,23,0.534884,0.502849,0.456134,0.810591,0.765378,0.812050
2,3874,5,3,15,NaN,NaN,NaN,NaN,NaN,NaN,...,3,1.000000,0.600000,3,0.120000,0.243186,0.456134,0.615402,0.196424,0.331868
3,3872,2,3,10,NaN,15.0,15.0,11.0,4.266667,0.0,...,7,1.000000,0.700000,13,0.928571,0.158300,0.456134,0.198938,0.224858,0.174435
4,3870,1,1,9,NaN,NaN,NaN,NaN,NaN,NaN,...,6,0.857143,0.666667,8,0.307692,0.182655,0.456134,0.161311,0.192835,0.167960
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1141713,3872,1,3,13,NaN,22.0,22.0,14.0,10.181818,2.0,...,0,0.000000,0.000000,0,0.000000,0.330838,0.456134,0.328747,0.329541,0.329874
1141714,3873,1,1,13,NaN,NaN,NaN,NaN,NaN,NaN,...,9,1.000000,0.818182,14,0.538462,0.130783,0.456134,0.650737,0.123888,0.142459
1141715,3872,5,3,9,NaN,6.0,6.0,3.0,2.000000,1.0,...,8,0.888889,0.800000,18,1.125000,0.158856,0.456134,0.167935,0.127028,0.152819
1141716,3870,2,3,22,NaN,3.0,3.0,3.0,2.666667,2.0,...,0,0.000000,0.000000,0,0.000000,0.330838,0.456134,0.328747,0.329541,0.329874


In [24]:
model_lgb_240 = gload('model_lgb_240')
sub['label'] = model_lgb_240.predict_proba(test_240[feature_cols])[:, 1]
sub['label']

0          0.116748
1          0.047661
2          0.019536
3          0.412376
4          0.002288
             ...   
1141713    0.014652
1141714    0.004251
1141715    0.011832
1141716    0.409985
1141717    0.012224
Name: label, Length: 1141718, dtype: float64

In [25]:
np.mean(sub['label'])

0.10373854250630118

In [26]:
sub.to_csv('./storage/result_240.txt', index=None, header=None, sep='\t')

## predict v4

In [ ]:
# load data
test = read_param('test')
sub = read_param('sub')
model_lgb_v4 = gload('model_lgb_v4')
feature_cols = gload('feature_cols')
len(feature_cols)

In [119]:
model_lgb_v4 = gload('model_lgb_v4')
sub['label'] = model_lgb_v4.predict_proba(test[feature_cols])[:, 1]
sub['label']

0          0.039237
1          0.001455
2          0.158848
3          0.399821
4          0.157442
             ...   
1141678    0.114890
1141679    0.485331
1141680    0.123951
1141681    0.056573
1141682    0.079274
Name: v4, Length: 1141683, dtype: float64

In [125]:
np.mean(sub['label'])

0.3678238

In [124]:
sub.to_csv('./storage/result_v4.txt', index=None, header=None, sep='\t')

## predict ensemble_v6

In [32]:
feature_cols = gload('feature_cols')
model_lgb_v4 = gload('model_lgb_v4')
model_nn_ensemble_v6 = gload('model_nn_ensemble_v6')
test = read_param('test')

pre1 = model_lgb_v4.predict_proba(test[feature_cols])[:, 1]
pre1 = pd.DataFrame(pre1)
pre1.columns = ['pre1']
x = pd.concat([pre1, c], axis=1)
x

,pre1,v6_1,v6_2,v6_3,v6_123,v6_1_2_3
0,0.320011,0.653088,0.650794,0.692645,0.680957,0.662520
1,0.064840,0.502849,0.456134,0.810591,0.765378,0.812050
2,0.033373,0.243186,0.456134,0.615402,0.196424,0.331868
3,0.384727,0.158300,0.456134,0.198938,0.224858,0.174435
4,0.015286,0.182655,0.456134,0.161311,0.192835,0.167960
...,...,...,...,...,...,...
1141713,0.031599,0.330838,0.456134,0.328747,0.329541,0.329874
1141714,0.017698,0.130783,0.456134,0.650737,0.123888,0.142459
1141715,0.060716,0.158856,0.456134,0.167935,0.127028,0.152819
1141716,0.492970,0.330838,0.456134,0.328747,0.329541,0.329874


In [33]:
ensemble_v6 = model_nn_ensemble_v6.predict(x, batch_size=8192)
ensemble_v6 = pd.DataFrame(ensemble_v6)
ensemble_v6.columns = ['ensemble_v6']
ensemble_v6

,ensemble_v6
0,0.735973
1,0.491332
2,0.033659
3,0.494165
4,0.014505
...,...
1141713,0.047441
1141714,0.008886
1141715,0.049934
1141716,0.758100


In [34]:
sub['label'] = ensemble_v6['ensemble_v6']

In [35]:
np.mean(sub['label'])

0.35808238

In [36]:
sub.to_csv('./storage/result_ensemble_v6.txt', index=None, header=None, sep='\t')

## output result

In [121]:
final_test

,v4,v5
0,0.039237,0.332870
1,0.001455,0.481971
2,0.158848,0.332870
3,0.399821,0.587821
4,0.157442,0.689065
...,...,...
1141678,0.114890,0.332870
1141679,0.485331,0.544487
1141680,0.123951,0.468212
1141681,0.056573,0.332870


In [123]:
model_5 = gload('model_5')
sub['label'] = model_5.predict(final_test, batch_size=8192)
sub['label']

0          0.062062
1          0.006476
2          0.282007
3          0.825728
4          0.506535
             ...   
1141678    0.204403
1141679    0.880721
1141680    0.350748
1141681    0.095675
1141682    0.072380
Name: label, Length: 1141683, dtype: float32

In [124]:
sub.to_csv('./storage/result.txt', index=None, header=None, sep='\t')

In [125]:
np.mean(sub['label'])

0.3678238

In [126]:
np.mean(final_test['v4'])

0.18850230499971676

In [17]:
# 压缩存储
for i in ['X_train_all', 'test', 'sub', 'feature_cols', 'data']:
    with read_param(i) as tmp:
        gdump(tmp, i)
    print(i)

AttributeError: __enter__